In [1]:
!pip install -U sentence_transformers numpy spacy tarfile

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/8a/08/a7e5dadc21fe193baea5f257e11b7b70cc27a89692fc9e3ed690e55cc4b6/numpy-1.26.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached numpy-1.26.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement tarfile (from versions: none)
ERROR: No matching distribution found for tarfile


In [1]:
import tarfile
import numpy as np
from sentence_transformers import SentenceTransformer
import spacy
import csv
import time

In [2]:
!python -m spacy download el_core_news_sm

/home/spyros/miniconda3/envs/llms/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
  Using cached https://github.com/explosion/spacy-models/releases/download/el_core_news_sm-3.7.0/el_core_news_sm-3.7.0-py3-none-any.whl (12.6 MB)
✔ Download and installation successful
You can now load the package via spacy.load('el_core_news_sm')


In [2]:
nlp = spacy.load('el_core_news_sm')

model = SentenceTransformer('lighteternal/stsb-xlm-r-greek-transfer')

In [8]:
print(model.device)

cuda:0


In [9]:
model = model.to('cuda')

In [10]:
import torch
print(torch.cuda.is_available())

True


In [11]:
def process_paragraph(paragraph):
    # Split the paragraph into sentences
    doc = nlp(paragraph)
    sentences = [sent.text for sent in doc.sents]

    # Generate an embedding for each sentence
    embeddings = model.encode(sentences)

    # Aggregate the embeddings by taking the mean
    paragraph_embedding = np.mean(embeddings, axis=0)

    return paragraph_embedding

In [12]:
def process_paragraphs(paragraphs):
    # Split each paragraph into sentences
    docs = list(nlp.pipe(paragraphs))
    sentences = [[sent.text for sent in doc.sents] for doc in docs]

    # Flatten the list of sentences
    sentences_flat = [sent for sublist in sentences for sent in sublist]

    # Generate an embedding for each sentence
    embeddings = model.encode(sentences_flat)

    # Reshape the embeddings to match the original structure
    embeddings = np.split(embeddings, np.cumsum([len(sents) for sents in sentences[:-1]]))

    # Aggregate the embeddings by taking the mean
    paragraph_embeddings = [np.mean(emb, axis=0) for emb in embeddings]

    return paragraph_embeddings


In [14]:
def process_raw_text_file(file_path, output_csv_path, batch_size=2048):
    # Open the output CSV file
    with open(file_path, 'r') as file, open(output_csv_path, 'w', newline='') as csvfile:
        # Create a CSV writer
        writer = csv.writer(csvfile)

        # Write the header row
        writer.writerow(['line_index', 'embedding'])


        start_time = time.time()
        paragraphs = []
        for line_index, line in enumerate(file):
            paragraph = line.strip()
            paragraphs.append(paragraph)
            if len(paragraphs) == batch_size:
                #process embeddings
                embeddings = process_paragraphs(paragraphs)
                for i, embedding in zip(range(line_index - batch_size + 1, line_index + 1), embeddings):
                    writer.writerow([i, embedding.tolist()])
                # Throughput
                end_time = time.time() 
                throughput = batch_size / (end_time - start_time) 
                print(f"Reached batch size at line {line_index}. Throughput: {throughput} lines/sec")
                paragraphs = []
                start_time = time.time() 
                
        if len(paragraphs):
            print("Final batch:")
            embeddings = process_paragraphs(paragraphs)
            for i, embedding in zip(range(line_index - len(paragraphs) + 1, line_index + 1), embeddings):
                writer.writerow([i, embedding.tolist()])

In [ ]:
process_raw_text_file('/home/spyros/Temp/output.txt', '/home/spyros/Temp/output-embeddings.csv')

Reached batch size at line 2047. Throughput: 30.514341876529585 lines/sec
Reached batch size at line 4095. Throughput: 30.057197049087193 lines/sec
Reached batch size at line 6143. Throughput: 33.612781425155376 lines/sec
Reached batch size at line 8191. Throughput: 28.469353700514873 lines/sec
Reached batch size at line 10239. Throughput: 30.396464465816603 lines/sec
Reached batch size at line 12287. Throughput: 27.36760373885477 lines/sec
Reached batch size at line 14335. Throughput: 27.63264322077476 lines/sec
Reached batch size at line 16383. Throughput: 28.790211494872732 lines/sec
Reached batch size at line 18431. Throughput: 25.859581260047946 lines/sec
Reached batch size at line 20479. Throughput: 28.968540003570205 lines/sec
Reached batch size at line 22527. Throughput: 29.59702877472612 lines/sec
Reached batch size at line 24575. Throughput: 29.64418201148695 lines/sec
Reached batch size at line 26623. Throughput: 28.412085288596103 lines/sec
Reached batch size at line 28671.